## Day 5, Part 2, Attempt 2

No more storing everything in a big old list

In [2]:
with open("./example.txt") as f:
    example_lines = []
    for line in f.readlines():
        example_lines.append(line.strip())

with open("./input.txt") as f:
    input_lines = []
    for line in f.readlines():
        input_lines.append(line.strip())

In [69]:
def part2(lines: list[str], verbose = True) -> int:
    seeds_input = list(map(int, lines[0].split(": ")[-1].split()))
    seeds_ranges = []
    for i in range(0, len(seeds_input), 2):
        seeds_ranges.append(
            (seeds_input[i], seeds_input[i]+seeds_input[i+1])
        )
    
    # Now have a list of the seed ranges in form [(Xs, Xe), (Ys, Ye), ...] where s=start, e=end

    # Ignore labels this time, just break the input up into blocks!
    blocks = []
    tmp = []
    for line in lines[3:]:
        if "map" in line:
            blocks.append(tmp)
            tmp = []
        elif not line:
            continue
        else:
            source, destination, length = list(map(int, line.split()))
            tmp.append(
                (source, destination, length)
            )

    # Now we have input blocks of the form
    # [map1=[(s1,d1,l1), (s2,d2,l2)], map2=[(s1,d1,l2)], ...]
    # So we can loop through these one at a time to tackle a mapping at a time.

    # Overlap types:
    # 1. none left, 2. overlap left, 
    # 3. overlap left right, 4. contained,
    # 5. overlap right, 6. none right
    # 7. perfect mapping

    for i, mappings in enumerate(blocks):
        if verbose: print(f"{i}/{len(blocks)}")
        if verbose: print(len(seeds_ranges))
        new_seeds_ranges = []
        while seeds_ranges:
            seed = seeds_ranges.pop()
            for destination, source, length in mappings:
                source_start, source_end = source, source+length - 1
                seed_start, seed_end = seed

                # 2. OVERLAP LEFT
                if source_start < seed_start and seed_start < source_end <= seed_end:
                    # so we can map from seed_start to source_end 
                    # but have to add source_end to seed_end to stack
                    seeds_ranges.append(
                        (source_end + 1, seed_end)
                    )
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (seed_start + source_dest_diff, source_end + source_dest_diff)
                    )
                    break

                # 3. OVERLAP LEFT AND RIGHT
                elif source_start < seed_start and source_end > seed_end:
                    # seed is fully contained within the mappable range, so we
                    # can map it and not worry any further!

                    # JUST MAP THE MIDDLE
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (seed_start + source_dest_diff, seed_end + source_dest_diff)
                    )
                    break

                # 4. COMPLETELY CONTAINED
                elif source_start >= seed_start and source_end <= seed_end:
                    # have to add the unmapped bits to left and right of seed to stack
                    seeds_ranges.append(
                        (seed_start, source_start - 1)
                    )
                    seeds_ranges.append(
                        (source_end + 1, seed_end)
                    )

                    # Map what we can of the source
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (source_start + source_dest_diff, source_end + source_dest_diff)
                    )
                    break

                # 5. OVERLAP RIGHT
                elif seed_start <= source_start < seed_end and source_end > seed_end:
                    # can map from source_start to seed_end, but have to 
                    # add seed_start to source_start to the stack as it is unmapped
                    seeds_ranges.append(
                        (seed_start, source_start - 1)
                    )
                    source_dest_diff = destination - source_start
                    new_seeds_ranges.append(
                        (source_start + source_dest_diff, seed_end + source_dest_diff)
                    )
                    break
                else:
                    # Situation 1. or 6. with no overlaps
                    assert source_end < seed_start or source_start > seed_end, f"{seed}, {(source_start, source_end)}"
            else:
                # if we didn't go into an if/elif in the for loop it goes to else  (never found a mapping -> identity)
                new_seeds_ranges.append(seed)
        
        seeds_ranges = new_seeds_ranges

    if verbose: print(f"{len(blocks)}/{len(blocks)}")
    

    # Ok now it's quite dodgy I don't know where the 0s are coming from and why the bottom range is less than top range in some.......... this wasnt the case but not sure what I changed :)
    LHSs = [lower_seed for lower_seed, _ in seeds_ranges if lower_seed != 0]
    print([(ls, rs) for ls, rs in sorted(seeds_ranges)])

    return min(LHSs)

assert part2(example_lines, verbose=False) == 46
part2(input_lines)  # 96937421, 3294143, 3294142, 50007219 is WRONG don't submit it AGAIN

[(46, 56), (64, 64), (65, 64), (77, 76), (78, 80), (82, 85), (90, 99)]
0/6
10
1/6
32
2/6
57
3/6
97
4/6
124
5/6
185
6/6
[(0, -1), (0, -1), (3294143, 3294142), (50007219, 50007218), (55894994, 55894993), (96937421, 101289596), (112720182, 112720181), (147158930, 148895271), (195723323, 195723322), (205128414, 212418126), (210934675, 210934674), (236362652, 236362651), (237484205, 239335676), (239335677, 240219804), (247133562, 247133561), (291726849, 291726848), (357336699, 357336698), (382939977, 382939976), (396950205, 396950204), (425139215, 425139214), (454883346, 460949968), (460949969, 489892966), (489892967, 491423224), (491423225, 498013578), (498013579, 507143542), (501359774, 501359773), (513152422, 513152421), (523874383, 523874382), (558378079, 594041600), (594041601, 629186938), (629186939, 646363830), (654039092, 654039091), (670368570, 726724169), (726724170, 761850944), (761850945, 776987442), (775918061, 775918060), (776987443, 786297951), (999993949, 1012172715), (10021

3294143

In [72]:
with open("./kate_input.txt") as f:
    kate_lines = []
    for line in f.readlines():
        kate_lines.append(line.strip())

assert part2(kate_lines, verbose=False) == 1493866
print("hmmmmmmmm")

[(0, -1), (0, -1), (1493866, 10954308), (50142747, 87630969), (64536060, 64536059), (111047381, 186786961), (129467115, 129467114), (130123821, 130123820), (182958962, 182958961), (186786962, 196534502), (217965045, 234010996), (227923036, 227923035), (237943150, 237943149), (243704176, 263927945), (263899862, 263899861), (276587655, 301646666), (301646667, 321931449), (304752282, 304752281), (329054675, 350239393), (356184002, 356184001), (369103719, 374370943), (369927146, 369927145), (372090746, 372090745), (374370944, 407052038), (407052039, 442539891), (442539892, 446320344), (442596292, 442596291), (450208338, 493814561), (490197617, 490197616), (493814562, 504806665), (504806666, 517754979), (511277888, 511277887), (517754980, 529961916), (592546766, 592546765), (604889911, 620089418), (624347933, 624347932), (624772153, 624772152), (648288351, 648288350), (665620049, 665620048), (732386637, 767331453), (798790248, 798790247), (813316060, 843958426), (838440546, 838440545), (843